# Logging Data With the Moku
-----
In this example notebook, we will show how to log data with the Moku, building up to writing a short sequencer script. In order to log data, we will use the `LogData` class from `splendaq`, which pulls from the Moku Python API.

In [ ]:
from splendaq.daq import LogData
from splendaq.io import convert_li_to_h5

import numpy as np

moku_ip = 'your_ip_address'

## Simple Data Logging

### The `LogData` Context Manager

The `LogData` class should be interacted with as a context manager, as it properly closes the connection to the Moku once complete. When entering the context manager, the user can set various global settings for the Moku, e.g. the digitization rate and the maximum number of seconds to save to a single file. Below, we show the docstring of the class for the user's benefit. Note: the `force_connect` argument should be set to `True` as a last resort, otherwise you might be kicking another user off of the Moku!

In [ ]:
?LogData

### Setting DC Values and Logging Data

For a simple example, we show below how to log data from Input 1, where we are supplying a some DC voltage from Output 1. We'll log 30 seconds of data, where we are saving the data being read out by Input 1.

In [ ]:
with LogData(moku_ip) as LOG:
    LOG.set_input_channels(1)
    dc_settings = LOG.dc_settings(
        dc_value=0.1,
    )
    LOG.set_output_channel(1, 'DC', **dc_settings)
    LOG.log_data(30)

This will have saved a file with the `.li` extension, a Liquid Instruments binary file. In order to convert it to a `splendaq` HDF5 file, we can use `splendaq.io.convert_li_to_h5`. The subsequent file can then be loaded with `splendaq.io.Reader`.

In [ ]:
convert_li_to_h5('created_file.li', my_os='macos')

### Setting Waveforms

There are various waveforms that can be outputted by the Moku (beyond a simple DC value), they are:
 - Sine wave
 - Square wave
 - Ramp
 - Pulse

As with the DC values, these various waveforms settings can be accessed and then set via the `set_output_channel` method. As an example, we show setting up data taking with a roughly square pulse. This time, let's log input channels 1 and 2, which can be done by instead passing a list to `set_input_channels`.

In [ ]:
with LogData(moku_ip) as LOG:
    LOG.set_input_channels([1, 2])
    pulse_settings = LOG.pulse_settings(
        amplitude=0.1,
        frequency=10,
        offset=0,
        edge_time=50e-6,
        pulse_width=1e-3,
    )
    LOG.set_output_channel(1, 'Pulse', **pulse_settings)

    LOG.log_data(30)

The various waveform settings can be accessed via:
 - `splendaq.daq.LogData.dc_settings`
 - `splendaq.daq.LogData.pulse_settings`
 - `splendaq.daq.LogData.ramp_settings`
 - `splendaq.daq.LogData.sine_settings`
 - `splendaq.daq.LogData.square_settings`
 
 Check their docstrings to see the different tunable parameters and allowed values!

## Writing a Short Sequencer

Now that we have the ability to log single datasets, let's see how we could write a short sequencer that would automatically save data at different specified settings. A common measurement is the IV curve: how does some device respond to DC changes in voltage?

In [ ]:
dc_values = np.linspace(0, 1, num=10)

for dcval in dc_values:
    with LogData(moku_ip) as LOG:
        LOG.set_input_channels(1)
        dc_settings = LOG.dc_settings(
            dc_value=dcval,
        )
        LOG.set_output_channel(1, 'DC', **dc_settings)
        LOG.log_data(30, file_name_prefix=f"splendaq_iv_{dcval}")

What if we have a system that has two voltages that we can change? This would, for example, apply in the case of characterizing a HEMT (high electron mobility transistor), where we change both drain-to-source and gate-to-source voltages in the common source configuration. We could write a sequencer in a very similar way as the previous one, such as below

In [ ]:
vgs_values = np.linspace(0, 1, num=10)
vds_values = np.linspace(0, 1, num=10)

for vgs in vgs_values:
    for vgs in vgs_values:
        with LogData(moku_ip) as LOG:
            LOG.set_input_channels(1)

            vgs_settings = LOG.dc_settings(
                dc_value=vgs,
            )
            LOG.set_output_channel(1, 'DC', **vgs_settings)

            vds_settings = LOG.dc_settings(
                dc_value=vds,
            )
            LOG.set_output_channel(2, 'DC', **vds_settings)

            LOG.log_data(30, file_name_prefix=f"splendaq_iv_{vgs}_{vds}")